# Pysat para horarios escolares

In [3]:
%pip install python-sat
%pip install matplotlib
%pip instal pprint

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "instal" - maybe you meant "install"



### Função para montar a tabela:

In [4]:
import matplotlib.pyplot as plt
import numpy as np


def plot_timetable(timetable, target_class):
    fig, ax = plt.subplots(figsize=(10, 6))

    # Set the title, x-axis labels, and y-axis labels
    ax.set_title(f'Timetable for {target_class}')
    ax.set_xticks(np.arange(len(days)))
    ax.set_yticks(np.arange(len(hours)))
    ax.set_xticklabels(days)
    ax.set_yticklabels(hours)

    # Create a grid to separate the cells
    ax.set_xticks(np.arange(-0.5, len(days), 1), minor=True)
    ax.set_yticks(np.arange(-0.5, len(hours), 1), minor=True)
    ax.grid(which='minor', linestyle='-', linewidth=1)

    # Loop through the timetable and display the subject and teacher for the target class
    for item in timetable:
        c, t, s, d, h = item
        if c == target_class:
            day_idx = days.index(d)
            hour_idx = hours.index(h)
            text = f"{s}\n{t}"
            ax.text(day_idx, hour_idx, text, ha='center', va='center', fontsize=10)

    plt.show()

In [ ]:
from pysat.formula import CNF
from pysat.solvers import Solver
from pprint import pprint

# Input data
classes = ['class1', 'class2', 'class3']
teachers = ['teacher1', 'teacher2', 'teacher3']
subjects = ['subject1', 'subject2', 'subject3', 'alg1', 'calc1', 'logica']
days = ['mon', 'tue', 'wed', 'thu', 'fri']
hours = ['7am', '8am', '9am', '10am', '11am']

teacher_subjects = {
    'teacher1': ['subject1', 'subject3', 'logica', 'calc1'],
    'teacher2': ['subject2', 'subject3', 'alg1'],
    'teacher3': ['subject1', 'subject2']
}


# Create variable map
var_map = {}
counter = 1
for c in classes:
    for t in teachers:
        for s in subjects:
            for d in days:
                for h in hours:
                    var_map[(c, t, s, d, h)] = counter
                    counter += 1

# Helper function to get variable id
def var(c, t, s, d, h):
    return var_map[(c, t, s, d, h)]

# Initialize CNF formula
cnf = CNF()

# Constraint 1: Each class has exactly one subject at each time slot
for c in classes:
    for d in days:
        for h in hours:
            # At least one subject
            cnf.append([var(c, t, s, d, h) for t in teachers for s in subjects])
            # At most one subject
            for s1 in subjects:
                for s2 in subjects:
                    if s1 != s2:
                        for t in teachers:
                            cnf.append([-var(c, t, s1, d, h), -var(c, t, s2, d, h)])

# Constraint 2: Each teacher teaches at most one class at a time
for t in teachers:
    for d in days:
        for h in hours:
            for c1 in classes:
                for c2 in classes:
                    if c1 != c2:
                        for s1 in subjects:
                            for s2 in subjects:
                                cnf.append([-var(c1, t, s1, d, h), -var(c2, t, s2, d, h)])



# Constraint 3: Each teacher can only teach the subjects they are capable of teaching
for c in classes:
    for t in teachers:
        for s in subjects:
            if s not in teacher_subjects[t]:
                for d in days:
                    for h in hours:
                        cnf.append([-var(c, t, s, d, h)])


# Solve the CNF formula and visualize the timetable for each class
with Solver(name='cd', bootstrap_with=cnf) as solver:
    if solver.solve():
        model = solver.get_model()
        timetable = []
        for c in classes:
            for t in teachers:
                for s in subjects:
                    for d in days:
                        for h in hours:
                            if model[var(c, t, s, d, h) - 1] > 0:
                                timetable.append((c, t, s, d, h))

        # Generate a timetable for each class
        for c in classes:
            plot_timetable(timetable, c)
    else:
        print("No solution found")
